In [6]:
import pandas as pd
import datetime 
import re, string, unicodedata
import nltk 
import contractions

In [2]:
with open('sample.srt', 'r') as file:
    text = file.read()

subtime = []
lines = []
prev_line = ''
line_nm = 0

for line in text.split('\n'):
    if line == '':
        if prev_line != '': 
            lines.append(prev_line)
        prev_line = ''
        line_nm = 0
    else:
        if line_nm == 1:
            subtime.append(line)
        if line_nm == 2: 
            prev_line = line
        elif line_nm > 2: 
            prev_line += ' ' + line 
        line_nm += 1

In [11]:
time_on = []
time_off = []

for line in subtime:
    x = line.split(' ')
    on = x[0].replace(',','.')
    off = x[2].replace(',','.')
    time_on.append(on)
    time_off.append(off)
    

In [27]:
sub_len = []
for line in range(len(time_on)):
    dt = datetime.datetime.strptime(time_off[line], "%H:%M:%S.%f") - datetime.datetime.strptime(time_on[line], "%H:%M:%S.%f")
    secs = dt.total_seconds()
    sub_len.append(secs)

In [29]:
df = pd.DataFrame()
df['time_on'] = time_on
df['time_off'] = time_off
df['time_length'] = sub_len
df['text'] = lines

In [84]:
def rm_btw_sq_brackets(text):
    """
    '\' - allows escaping special character coding, to match special characters exactly
    
    """
    return re.sub('\[[^]]*\]', '', text) #delete text inside square brackets (~ stage directions)

def rm_btw_asterisk(text):
    return re.sub('[\*].*?[\*]', '', text)

def replace_contractions(text):
    return contractions.fix(text)

def denoise(text):
    text = rm_btw_sq_brackets(text)
    text = rm_btw_asterisk(text)
    text = replace_contractions(text)
    return text

In [85]:
cleantxt = []
for line in range(len(df['text'])):
    cleantxt.append(denoise(df['text'].iloc[line]))

df['denoised_text'] = cleantxt
print(df.to_string())

           time_on      time_off  time_length                                               text                                      denoised_text
0     00:00:36.937  00:00:40.574        3.637                                   ** [ Whistling ]                                                   
1     00:00:41.509  00:00:44.712        3.203                      [ Pencil Scribbling Rapidly ]                                                   
2     00:00:44.778  00:00:47.515        2.737                         ** [ Whistling Continues ]                                                   
3     00:01:04.598  00:01:07.801        3.203  [ Man Narrating ]   This is a story   of boy m...               This is a story   of boy meets girl.
4     00:01:12.273  00:01:15.709        3.436      The boy, Tom Hansen   of Margate, New Jersey,      The boy, Tom Hansen   of Margate, New Jersey,
5     00:01:15.776  00:01:18.579        2.803  grew up believing that   he'd never truly be h...  grew up believ

In [87]:
df.to_pickle('processed_sub.pkl')